In [13]:
import pandas as pd


pd.set_option("display.max_columns", None)   # mostra todas as colunas
pd.set_option("display.width", None)         # evita quebra de linha
pd.set_option("display.max_rows", None)      # idem para linhas, se quiser

In [14]:
df = pd.read_csv('bench_results.csv')
df.head()

,Format,Status❔,Size (MB),Inference Time(ms/im),Avarage Processing Time (ms/im),FPS,Throughput (img/s),cpu_proc_avg (%),ram_proc_mb (MB),gpu_avg (%),power_avg (W),ram_total_mb (MB),metrics/mAP50-95(M),metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),device
0,TensorRT FP32,✅,40.5,10.84,22.40,44.65,43.83,39.63,2394.6,54.27,0.0,12078.0,0.5999,0.9547,0.9521,0.9884,0.9239,0.898,0.9437,JetsonOrin16gb
1,TensorRT FP16,✅,40.5,10.83,22.16,45.13,44.29,59.28,2454.8,57.27,0.0,12472.8,0.5999,0.9547,0.9521,0.9884,0.9239,0.898,0.9437,JetsonOrin16gb
2,ONNX,❌ UnboundLocalError,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,local variable 'fmt' referenced before assignment,JetsonOrin16gb
3,PyTorch,❌ UnboundLocalError,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,local variable 'fmt' referenced before assignment,JetsonOrin16gb


In [8]:
from ultralytics.utils.benchmarks import benchmark
from ultralytics import YOLO

results_ultr = benchmark(model=f"models/land-seg.pt", data="dataset/landslide_dataset_1000/data.yml", imgsz=512, format='-', device =0)






Setup complete ✅ (8 CPUs, 15.3 GB RAM, 36.0/115.4 GB disk)

Benchmarks complete for models/land-seg.pt on dataset/landslide_dataset_1000/data.yml at imgsz=512 (8.57s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
    Format Status❔  Size (MB)  metrics/mAP50-95(M)  Inference time (ms/im)    FPS
0  PyTorch       ✅       19.6               0.6295                   24.78  40.35



In [ ]:
results_ultr = benchmark(model=f"models/land-seg.pt", data="dataset/landslide_dataset_1000/data.yml", imgsz=512, format='engine', device =0)

Setup complete ✅ (8 CPUs, 15.3 GB RAM, 36.0/115.4 GB disk)

Benchmarks complete for models/land-seg.pt on dataset/landslide_dataset_1000/data.yml at imgsz=512 (284.69s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
     Format Status❔  Size (MB)  metrics/mAP50-95(M)  Inference time (ms/im)    FPS
0  TensorRT       ✅       41.0               0.5998                   27.15  36.83



In [ ]:
results_ultr = benchmark(model=f"models/land-seg.pt", data="dataset/landslide_dataset_1000/data.yml", imgsz=512, format='onnx', device =0)

Setup complete ✅ (8 CPUs, 15.3 GB RAM, 36.0/115.4 GB disk)

Benchmarks complete for models/land-seg.pt on dataset/landslide_dataset_1000/data.yml at imgsz=512 (43.07s)
Benchmarks legend:  - ✅ Success  - ❎ Export passed but validation failed  - ❌️ Export failed
  Format Status❔  Size (MB)  metrics/mAP50-95(M)  Inference time (ms/im)  FPS
0   ONNX       ✅       38.6               0.5998                  153.89  6.5



In [9]:
m = YOLO("models/land-seg.pt")
m.export(format="engine", device=0, half=True, dynamic=False, nms=True)  # FP16 + estático + NMS no engine (quando suportado)

Ultralytics 8.3.161 🚀 Python-3.10.12 torch-2.5.0a0+872d972e41.nv24.08 CUDA:0 (Orin, 15656MiB)
YOLO11s-seg summary (fused): 113 layers, 10,067,203 parameters, 0 gradients, 35.3 GFLOPs

PyTorch: starting from 'models/land-seg.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) ((1, 300, 38), (1, 32, 128, 128)) (19.6 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.58...
ONNX: export success ✅ 3.9s, saved as 'models/land-seg.onnx' (38.7 MB)

TensorRT: starting export with TensorRT 10.3.0...
[09/12/2025-06:52:45] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 1516, GPU 9207 (MiB)
[09/12/2025-06:52:46] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +926, GPU +711, now: CPU 2442, GPU 9918 (MiB)
[09/12/2025-06:52:46] [TRT] [I] ----------------------------------------------------------------
[09/12/2025-06:52:46] [TRT] [I] Input filename:   models/land-seg.onnx
[09/12/2025-06:52:46] [TRT] [I] ONNX IR version:  0.0.

'models/land-seg.engine'

In [10]:
from benchmark_PI import benchmark_PI
results = benchmark_PI(model="models/land-seg.pt", data="dataset/landslide_dataset_1000/data.yml", imgsz=512, format='engine', device =0)

Verificando necessidade de Exportar o Modelo

Modelo Já exportado, pulando exportação.
Aquecendo o modelo

Loading /home/jetson/Documents/YOLO_tests/models/land-seg.engine for TensorRT inference...
[09/12/2025-06:59:21] [TRT] [I] Loaded engine size: 23 MiB
[09/12/2025-06:59:21] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +12, now: CPU 0, GPU 31 (MiB)
Aquecimento concluído

Iniciando Monitor_GPU
Testando com 212 imagens!
Ultralytics 8.3.161 🚀 Python-3.10.12 torch-2.5.0a0+872d972e41.nv24.08 CUDA:0 (Orin, 15656MiB)
Loading /home/jetson/Documents/YOLO_tests/models/land-seg.engine for TensorRT inference...
[09/12/2025-06:59:25] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[09/12/2025-06:59:25] [TRT] [I] Loaded engine size: 23 MiB
[09/12/2025-06:59:25] [TRT] [

val: Scanning /home/jetson/Documents/YOLO_tests/dataset/landslide_dataset_1000/valid/labels.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 212/212 [00:03<00:00, 56.13it/s]

                   all        212       3029      0.954      0.953      0.987      0.969      0.928      0.902      0.952      0.636
Speed: 0.7ms preprocess, 6.9ms inference, 0.0ms loss, 1.8ms postprocess per image



A task é segment, métrica principal: metrics/mAP50-95(M)



In [ ]:
print(results)

[{'Format': 'TensorRT', 'Status❔': '✅', 'Size (MB)': 23.2, 'Inference Time(ms/im)': 6.66, 'Avarage Processing Time (ms/im)': 17.15, 'FPS': 58.32, 'Throughput (img/s)': 56.79, 'cpu_proc_avg (%)': 83.85, 'ram_proc_mb (MB)': 3163.9, 'gpu_avg (%)': 46.91, 'power_avg (W)': 0.0, 'ram_total_mb (MB)': 10244.0, 'metrics/mAP50-95(M)': 0.6357, 'metrics/precision(B)': 0.954, 'metrics/recall(B)': 0.9525, 'metrics/mAP50(B)': 0.9874, 'metrics/precision(M)': 0.9277, 'metrics/recall(M)': 0.9022, 'metrics/mAP50(M)': 0.9523}]
